In [ ]:
import cv2
import numpy as np
from IPython.display import display
from PIL import Image
from src.config import CONFIG_DEFAULTS as config
from src.logger import logger
min_matching_threshold =  0.3
max_matching_variation = 0.41
marker_rescale_range =  (35, 100)
marker_rescale_steps =  10
apply_erode_subtract =  0
def order_points(pts):
    rect = np.zeros((4, 2), dtype="float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect
#used this fucthon to crop based four point of four side of paper that devide based maker
def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width of the new image, which will be the
    width_a = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    width_b = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))

    max_width = max(int(width_a), int(width_b))
    # max_width = max(int(np.linalg.norm(br-bl)), int(np.linalg.norm(tr-tl)))

    # compute the height of the new image, which will be the
    height_a = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    height_b = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    max_height = max(int(height_a), int(height_b))
    # max_height = max(int(np.linalg.norm(tr-br)), int(np.linalg.norm(tl-br)))

    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array(
        [
            [0, 0],
            [max_width - 1, 0],
            [max_width - 1, max_height - 1],
            [0, max_height - 1],
        ],
        dtype="float32",
    )

    transform_matrix = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, transform_matrix, (max_width, max_height))

    # return the warped image
    return warped
def normalize_util(img, alpha=0, beta=255):
    return cv2.normalize(img, alpha, beta, norm_type=cv2.NORM_MINMAX)
def resize_util_h(img, u_height, u_width=None):
        if u_width is None:
            h, w = img.shape[:2]
            u_width = int(w * u_height / h)
        return cv2.resize(img, (int(u_width), int(u_height)))    
def resize_util(img, u_width, u_height=None):
        if u_height is None:
            h, w = img.shape[:2]
            u_height = int(h * u_width / w)
        return cv2.resize(img, (int(u_width), int(u_height)))
marker = cv2.imread("savedmar.jpeg", cv2.IMREAD_GRAYSCALE)
def getBestMatch(image_eroded_sub):

        descent_per_step = (
            marker_rescale_range[1] - marker_rescale_range[0]
        ) // marker_rescale_steps
        _h, _w = marker.shape[:2]
        res, best_scale = None, None
        all_max_t = 0

        for r0 in np.arange(marker_rescale_range[1],marker_rescale_range[0],-1 * descent_per_step,):  # reverse order
            s = float(r0 * 1 / 100)
            if s == 0.0:
                continue
            rescaled_marker = resize_util_h(marker, u_height=int(_h * s))
            # res is the black image with white dots
            res = cv2.matchTemplate(image_eroded_sub, rescaled_marker, cv2.TM_CCOEFF_NORMED )
            # cv2.imshow(str(5+r0),res)
            max_t = res.max()
            if all_max_t < max_t:
               # print("-------------------------------------------------------------\n")
                #print('Scale: '+str(s)+', Circle Match: '+str(round(max_t*100,2))+'%')
                #print("-------------------------------------------------------------\n")
                best_scale, all_max_t = s, max_t
        return best_scale, all_max_t
def apply_filter(image):
        
       
        image_eroded_sub = normalize_util(
            image
            if apply_erode_subtract
            else 
            (image - cv2.erode(image, kernel=np.ones((5, 5)), iterations=5))
        )
 # Quads on warped image
        quads = {}
        h1, w1 = image_eroded_sub.shape[:2]
        midh, midw = h1 // 3, w1 // 2
        origins = [[0, 0], [midw, 0], [0, midh], [midw, midh]]
        quads[0] = image_eroded_sub[0:midh, 0:midw]
        quads[1] = image_eroded_sub[0:midh, midw:w1]
        quads[2] = image_eroded_sub[midh:h1, 0:midw]
        quads[3] = image_eroded_sub[midh:h1, midw:w1]

        # Draw Quadlines
        image_eroded_sub[:, midw : midw + 2] = 255
        image_eroded_sub[midh : midh + 2, :] = 255
        # cv2.imshow("e+1",image_eroded_sub)
        display(Image.fromarray(image_eroded_sub))

        best_scale, all_max_t = getBestMatch(image_eroded_sub)
        # cv2.imshow("e+2",all_max_t)
        # if best_scale is None:
        #     # TODO: Plot and see performance of marker_rescale_range
        #     if config.outputs.show_image_level >= 1:
        #         MainOperations.show("Quads", image_eroded_sub)
        #     return None
        print(all_max_t)
        optimal_marker = resize_util_h(
            marker, u_height=int(marker.shape[0] * best_scale)
        )
      
        _h, w = optimal_marker.shape[:2]
        centres = []
        sum_t, max_t = 0, 0
        logger.info("Matching Marker:\t", end=" ")
        for k in range(0, 4):
            res = cv2.matchTemplate(quads[k], optimal_marker, cv2.TM_CCOEFF_NORMED)
            
            max_t = res.max()
            logger.info(f"Quarter{str(k + 1)}: {str(round(max_t, 3))} ", end="\t")
            # if (
            #     max_t < self.min_matching_threshold
            #     or abs(all_max_t - max_t) >= self.max_matching_variation
            # ):
            #     # Warning - code will stop in the middle. Keep Threshold low to
            #     # avoid.
            #     logger.error(
            #         args["current_file"].name,
            #         "\nError: No circle found in Quad",
            #         k + 1,
            #         "\n\t min_matching_threshold",
            #         self.min_matching_threshold,
            #         "\t max_matching_variation",
            #         self.max_matching_variation,
            #         "\t max_t",
            #         max_t,
            #         "\t all_max_t",
            #         all_max_t,
            #     )
            #     if config.outputs.show_image_level >= 1:
            #         MainOperations.show(
            #             "no_pts_" + args["current_file"].name, image_eroded_sub, 0
            #         )
            #         MainOperations.show(
            #             "res_Q" + str(k + 1) + " (" + str(max_t) + ")", res, 1
            #         )
            #     return None

            pt = np.argwhere(res == max_t)[0]
            pt = [pt[1], pt[0]]
            pt[0] += origins[k][0]
            pt[1] += origins[k][1]
            print(">>",pt)
            image = cv2.rectangle(
                image, tuple(pt), (pt[0] + w, pt[1] + _h), (150, 150, 150), 2
            )
            # cv2.imshow(str(k + 5),image)
            # display:
            image_eroded_sub = cv2.rectangle(
                image_eroded_sub,
                tuple(pt),
                (pt[0] + w, pt[1] + _h),
                (50, 50, 50) if apply_erode_subtract else (155, 155, 155),
                4,
            )
            centres.append([pt[0] + w / 2, pt[1] + _h / 2])
            sum_t += max_t
        logger.info("Optimal Scale:", best_scale)
        # analysis data
        # threshold_circles.append(sum_t / 4)

        image = four_point_transform(image, np.array(centres))
        # appendSaveImg(1,image_eroded_sub)
        # appendSaveImg(1,image_norm)
        #  append_save_img(2, image_eroded_sub)
        # Debugging image -
        # res = cv2.matchTemplate(image_eroded_sub,optimal_marker,cv2.TM_CCOEFF_NORMED)
        # res[ : , midw:midw+2] = 255
        # res[ midh:midh+2, : ] = 255
        # show("Markers Matching",res)
        # if config.outputs.show_image_level >= 2 and config.outputs.show_image_level < 4:
        #     image_eroded_sub = ImageUtils.resize_util_h(
        #         image_eroded_sub, image.shape[0]
        #     )
        #     image_eroded_sub[:, -5:] = 0
        #     h_stack = np.hstack((image_eroded_sub, image))
        #     MainOperations.show(
        #         "Warped: " + args["current_file"].name,
        #         ImageUtils.resize_util(
        #             h_stack, int(config.dimensions.display_width * 1.6)
        #         ),
        #         0,
        #         0,
        #         [0, 0],
        #     )
        # iterations : Tuned to 2.
        # image_eroded_sub = image_norm - cv2.erode(image_norm, kernel=np.ones((5,5)),iterations=2)
        # cv2.imshow("e+",image)
        return image


image = cv2.imread("savedimage.jpeg", cv2.IMREAD_GRAYSCALE)
display(Image.fromarray(image))
display(Image.fromarray(apply_filter( image)))